In [5]:
# importing the dependencies
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.model_selection import GridSearchCV,cross_val_score

from sklearn.metrics import roc_auc_score
import pickle

In [6]:
# Loading the dataset
df = pd.read_pickle('../DataFrames/Scaled_label_encoded')

In [7]:
# seperating only the train set
train_set,test_set =  df.iloc[:28322],df.iloc[28322:]

In [8]:
# Making X and y
X = train_set.drop(columns=['label','id'],axis=1)
y = train_set['label']
X = X.iloc[:,[10,4,8,12,6,17,7,5,1,18,13,11,14,0]]

In [9]:
X.shape, y.shape

((28322, 14), (28322,))

In [10]:
X.head()

,s69,s16,s52,s71,s18,n5,s48,s17,s11,n6,n1,s70,n2,gender
0,2,3,1,1,1,-1.276756e-15,0,3,1,0.199507,1.141241,3,0.551897,1
1,2,3,1,1,1,1.110223e-16,1,3,1,-0.291835,-0.725895,3,-1.260755,1
2,2,3,1,1,1,4.579670e-16,0,3,1,-0.279014,-0.783598,3,-0.888190,1
3,2,3,1,1,1,1.082467e-15,0,3,1,-0.810034,-1.105319,3,-1.388074,0
4,3,1,2,3,3,1.096345e-15,1,3,0,0.068661,0.899080,3,-0.531002,1


In [11]:
y.head()

0    0.0
1    1.0
2    0.0
3    0.0
4    0.0
Name: label, dtype: float64

Making Train,Test,split

In [12]:
X_train,X_test,y_train,y_test = train_test_split(
    X,y,test_size=0.3,random_state=42,shuffle=True
)

Gonna use SKF to preserve the class Distribution

In [13]:
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
Dtree = DecisionTreeClassifier(random_state=42,max_depth=5)

In [14]:
Dtree.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

In [15]:
cross_val_score(Dtree,X_train,y_train,cv=skf,scoring='roc_auc').mean()

0.8399837366042752

In [16]:
# accuracy on the test set
Dtree_pred = Dtree.predict(X_test)
roc_auc_score(y_test,Dtree_pred)

0.7046147465610523

In [17]:
from joblib import load,dump
dump(Dtree,'../Saved Model/Dtree.joblib')

['../Saved Model/Dtree.joblib']